In [ ]:
!pip install -q transformers torch gradio

In [ ]:
# Imports
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr


# ✅ Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ✅ Load Models
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)


bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2
).to(device)


# ✅ Generate Fake News
def generate_fake_news(prompt):
    inputs = gpt2_tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = gpt2_model.generate(
        inputs,
        max_length=200,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        early_stopping=True
    )
    generated_text = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


# ✅ Detect Fake or Real News
def detect_news(text):
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    confidence = torch.softmax(logits, dim=1)[0][predicted_class].item()
    label = "🟥 Fake News" if predicted_class == 0 else "🟩 Real News"
    return f"{label} (Confidence: {confidence:.2f})"


# ✅ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 📰 Fake News Generator & Detector (GPT-2 + BERT)")
    with gr.Tab("🛠️ Generate Fake News"):
        with gr.Row():
            input_text = gr.Textbox(label="Enter a News Headline or Prompt", placeholder="e.g. A mysterious object was spotted in the sky...", lines=2)
        generate_btn = gr.Button("Generate")
        output_text = gr.Textbox(label="Generated News Article")
        generate_btn.click(generate_fake_news, inputs=input_text, outputs=output_text)


    with gr.Tab("🔍 Detect Fake or Real"):
        with gr.Row():
            detect_input = gr.Textbox(label="Enter a News Article or Statement", placeholder="Paste a paragraph to detect if it's fake or real...", lines=5)
        detect_btn = gr.Button("Detect")
        detect_output = gr.Textbox(label="Detection Result")
        detect_btn.click(detect_news, inputs=detect_input, outputs=detect_output)


# ✅ Launch
demo.launch()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0136a5631cce199a36.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
